# Project 101
### PARTE 2.3 - Modelos Redes Neuronales

### Contenido
- [Modelado](#modelos)

    - [**2.1. Machine Learning Simple**](#1)
        - [Dicky Fuller Test](#1.1)
        - [Autocorrelacion y Autocorrelacion Parcial](#1.2)
        - [SARIMAX model](#1.3)
     - [**2.2. Machine Learning Nixtla Conjunto**](#1)
     - [**2.1. Machine Learning Nixtla Conjunto + variable exogena**](#1)

    - [Resultados](#resultados)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import time
time.clock = time.time

from gluonts.dataset.pandas import PandasDataset
from gluonts.torch.model.deepar import DeepAREstimator

from sklearn.metrics import mean_squared_error, mean_absolute_error


c:\Users\Bravo15\anaconda3\envs\modelling-torch\lib\site-packages\gluonts\json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


__Importamos los datos__

In [2]:
df_impo_1= pd.read_csv('Data/Modelado/impo 1 - USA FLAT.csv')
df_impo_2= pd.read_csv('Data/Modelado/impo 2 - ORIENTE UPS.csv')
df_impo_3= pd.read_csv('Data/Modelado/impo 3 - CHINA LATIN LOGISTIC.csv')
df_impo_4= pd.read_csv('Data/Modelado/impo 4 - EUROPA UPS.csv')
df_impo_5= pd.read_csv('Data/Modelado/impo 5 - Courier Oriente FLAT Wish.csv')
df_impo_6= pd.read_csv('Data/Modelado/impo 6 - Impo Geobox Flat.csv')
df_impo_7= pd.read_csv('Data/Modelado/impo 7 - UPS MERCOSUR.csv')

df_expo_1= pd.read_csv('Data/Modelado/expo 1 - Exporta Simple - Puerta-Aeropuerto.csv')
df_expo_2= pd.read_csv('Data/Modelado/expo 2 - CARGA AEREA EXPO PREPAID.csv')
df_expo_3= pd.read_csv('Data/Modelado/expo 3 - CARGA AEREA EXPO - Q.csv')
df_expo_4= pd.read_csv('Data/Modelado/expo 4 - 4-Expo - Fedex Economy.csv')
df_expo_5= pd.read_csv('Data/Modelado/expo 5 - 6-Expo-UPS Express.csv')

In [3]:
comparison_tbl= pd.read_csv('Data/Modelado/comparison_tbl.csv')

__Funciones Utiles__

In [4]:
#para obtener la metrica de smape
def calculate_smape(actual_values, forecasted_values):
    absolute_percentage_errors = []
    
    for actual, forecasted in zip(actual_values, forecasted_values):
        numerator = abs(actual - forecasted)
        denominator = (abs(actual) + abs(forecasted)) + 1e-9  # Agregar una constante para evitar división por cero
        ape = numerator / (denominator / 2)
        absolute_percentage_errors.append(ape)
    
    smape = sum(absolute_percentage_errors) / len(absolute_percentage_errors)
    return smape

def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()

def unificar_df(lista_dataframe, lista_df):
    
    lista_df_clean= []
    n=0
    
    for df in lista_dataframe: 
        df[['family']]= lista_df[n][3:]
        dff = df[['family','ds', 'y']]
        lista_df_clean.append(dff)
        n+=1
    
    df_unificado= pd.concat(lista_df_clean)
    df_unificado=df_unificado[['ds', 'family', 'y']]
    
    return df_unificado

In [5]:
lista_df= ['df_impo_1', 'df_impo_2', 'df_impo_3', 'df_impo_4', 'df_impo_5','df_impo_6', 'df_impo_7',
           'df_expo_1', 'df_expo_2', 'df_expo_3', 'df_expo_4', 'df_expo_5']

lista_dataframe= [df_impo_1, df_impo_2, df_impo_3, df_impo_4, df_impo_5,df_impo_6, df_impo_7,
           df_expo_1, df_expo_2, df_expo_3, df_expo_4, df_expo_5]

df = unificar_df(lista_dataframe, lista_df)

In [6]:
df['ds']= pd.to_datetime(df['ds'])
df.head()

,ds,family,y
0,2019-01-06,impo_1,1950.0
1,2019-01-13,impo_1,2402.0
2,2019-01-20,impo_1,2782.0
3,2019-01-27,impo_1,3263.0
4,2019-02-03,impo_1,3843.0


#### 2.3.1 Glounts Deep Ar


Definimos el set de train y test

In [7]:
train = df.loc[df['ds'] < '2022-11-01']
valid = df.loc[df['ds'] >= '2022-11-01']

In [8]:
train_ds = PandasDataset.from_long_dataframe(train, target='y', item_id='family', 
                                       timestamp='ds', freq='W')

__Definicion del modelo DeepAr__

In [9]:
tic = time.clock()
estimator = DeepAREstimator(freq='W', #la frecuencia de los datos es semanal por lo que el modelo debe ser semanal tambien
                            context_length=4, # el modelo va a usar las ultimas 4 semanas para predecir la siguiente 
                            prediction_length=10, # va a predecir las proximas 10 semanas 
                            num_layers=3, #el modelo posee tres capas con un default de 40 nodos por capa 
                            dropout_rate= 0.2, #seteo de 20% de las units en una layer en cero de forma random, el default es 0.1
                            trainer_kwargs={ 'max_epochs':30}) #en vez de poner un early stopping defino un max epoch de 30

predictor = estimator.train(train_ds, num_workers=2)

toc =  time.clock() #frenamos el cronometro y a continuacion creamos la variable que guarda el tiempo transcurrido desde que empezo a correr el modelo hasta q termino
exetime = '{0:.4f}'.format(toc-tic)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Bravo15\anaconda3\envs\modelling-torch\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:71: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name  | Type        | Params | In sizes | Out sizes   
----------------------------------------------------------------
0 | model | DeepARModel | 36.9 K | ?        | [1, 100, 10]
----------------------------------------------------------------
36.9 K    Trainable params
0         Non-trainable params
36.9 K    Total params
0.148     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 7.31498 (best 7.31498), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\lightning_logs\\version_71\\checkpoints\\epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 6.96906 (best 6.96906), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\lightning_logs\\version_71\\checkpoints\\epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' reached 6.79262 (best 6.79262), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\lightning_logs\\version_71\\checkpoints\\epoch=2-step=150.ckpt' as top 1
Epoch 3, global step 200: 'train_loss' was not in top 1
Epoch 4, global step 250: 'train_loss' reached 6.71023 (best 6.71023), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\lightning_logs\\version_71\\checkpoints\\epoch=4-step=250.ckpt' as top 1
Epoch 

In [ ]:
pred = list(predictor.predict(train_ds))

all_preds = list()
for item in pred:
    family = item.item_id
    p = item.samples.mean(axis=0)
    p10 = np.percentile(item.samples, 10, axis=0)
    p90 = np.percentile(item.samples, 90, axis=0)
    p25 = np.percentile(item.samples, 25, axis=0)
    p75 = np.percentile(item.samples, 75, axis=0)
    dates = pd.date_range(start=item.start_date.to_timestamp(), periods=len(p), freq='W')
    family_pred = pd.DataFrame({'ds': dates, 
                                'family': family,
                                'pred': p,
                                'p25': p25,
                                'p75': p75,
                                'p10': p10, 
                                'p90': p90})
    all_preds += [family_pred]
all_preds = pd.concat(all_preds, ignore_index=True)
all_preds = all_preds.merge(valid, on=['ds', 'family'], how='left')

all_preds.dropna(inplace=True)

wmape(all_preds['y'], all_preds['pred'])

0.5394275519460465

In [ ]:
all_preds.head()

,ds,family,pred,p25,p75,p10,p90,y
0,2022-11-06,expo_1,1629.792847,1000.828369,2038.413666,765.331049,2527.628076,2720.4
1,2022-11-13,expo_1,1187.482544,846.443069,1660.420288,328.672028,2061.896509,290.1
2,2022-11-20,expo_1,1234.731201,873.337112,1732.976257,475.421011,2061.514209,906.0
3,2022-11-27,expo_1,1358.025635,938.338333,1793.695770,321.901254,2304.432690,2408.3
4,2022-12-04,expo_1,1328.474976,792.002075,1891.058716,-78.740099,2501.318140,326.0


__Resultados Obtenidos: Deep AR__

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go
import plotly.express as px
import dash_bootstrap_components as dbc

app = Dash(external_stylesheets=[dbc.themes.SLATE])



dropdown = dcc.Dropdown(
    id="ticker",
    options=[{"label": family, "value": family} for family in df["family"].unique()],
    value="impo_1",
    clearable=False, 
    style={'background-color': 'grey'}
)

app.layout = html.Div([
    html.H4('Deep AR Resultados'),
    html.P("Seleccionar motivo:"),
    dropdown,
    dcc.Graph(id="time-series-chart"),
])


@app.callback(
    Output("time-series-chart", "figure"), 
    Input("ticker", "value"))

def display_time_series(ticker):
    df_pivot = df.pivot(index='ds', columns='family', values='y')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)
    
    fig = px.line(df_pivot, x='ds', y=ticker, title=ticker, template='plotly_dark')
    fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)',
                      paper_bgcolor='rgba(0, 0, 0, 0)')
    
    #bandas de percentiles 
    p_ = all_preds.loc[all_preds['family'] == ticker]
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p10'], mode='lines', line=dict(color='rgba(255, 165, 0, 0.2)'), name='p10'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p90'], mode='lines', line=dict(color='rgba(255, 165, 0, 0.2)'), name='p90', fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p25'], mode='lines', line=dict(color='orange'), name='p25'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p75'], mode='lines', line=dict(color='orange'), name='p75', fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['pred'], mode='lines', line=dict(color='orange'), name='Forecast'))
    return fig


if __name__ == "__main__":
    app.run_server(debug=True, port=8080, mode='inline')

__Guardamos las métricas para cada una de las series__

In [ ]:
def guardar_metricas(comparison_tbl, all_preds, exetime):
    
    lista_family= []
    lista_mae=[]
    lista_rmse=[]
    lista_smape=[]
    lista_exetime=[]
    
    for fam in list(all_preds.family.unique()): 
        df_pred= all_preds[all_preds['family']== fam]
        actual_values= df_pred['y']
        predicted_values= df_pred['pred']
        
        mae = mean_absolute_error(actual_values, predicted_values)
        rmse = np.sqrt(mean_squared_error(actual_values, predicted_values))
        smape = calculate_smape(actual_values, predicted_values)
        
        lista_family.append(fam)
        lista_mae.append(mae)
        lista_rmse.append(rmse)
        lista_smape.append(smape)
        lista_exetime.append(exetime)
    
    raw_data = {'Serie': lista_family,
                'Modelo': 'Glounts Deep AR',
                'MAE':  lista_mae,
                'RMSE': lista_rmse,
                'sMAPE': lista_smape, 
                'Processing Time': lista_exetime}
    
    df_tbl = pd.DataFrame(raw_data)
    
    comparison_tbl = pd.concat([comparison_tbl, df_tbl], ignore_index=True)
    
    return comparison_tbl

In [ ]:
comparison_tbl=guardar_metricas(comparison_tbl, all_preds, exetime)

,Serie,Modelo,MAE,RMSE,sMAPE,Processing Time
130,expo_3,Glounts Deep AR,135.491901,146.753620,0.420255,0.1
131,expo_4,Glounts Deep AR,39.285555,55.438986,0.490382,0.1
132,expo_5,Glounts Deep AR,100.163306,107.438102,0.929222,0.1
133,impo_1,Glounts Deep AR,893.290283,976.204634,0.302469,0.1
134,impo_2,Glounts Deep AR,468.920086,511.354963,0.755665,0.1
135,impo_3,Glounts Deep AR,29.988889,39.672423,1.349579,0.1
136,impo_4,Glounts Deep AR,129.571404,132.943423,1.104780,0.1
137,impo_5,Glounts Deep AR,600.393624,913.997314,0.591386,0.1
138,impo_6,Glounts Deep AR,343.171177,426.975197,0.461060,0.1
139,impo_7,Glounts Deep AR,108.761310,118.138561,0.331779,0.1


#### 2.3.1 Glounts Deep AR - Variable exogena